In [1]:
import os

import litellm
from dotenv import load_dotenv
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.models.lite_llm import LiteLlm
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.adk.runners import InMemoryRunner
from google.genai import types
from IPython.display import Markdown, display

from agents_intensive_capstone.agents import (
    black_hat_factory,
    blue_hat_factory,
    green_hat_factory,
    white_hat_factory,
    yellow_hat_factory,
)

c:\Praveen\Personal\git\agents-intensive-capstone-2025\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## Configure Retry Options¶

In [2]:
load_dotenv()

# 2. Access the variable
api_key = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key setup complete.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.


In [3]:
litellm.use_litellm_proxy = True
gpt_model = LiteLlm(model="gpt-oss-20b")

In [4]:
gemini_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

In [5]:
# blue_hat = blue_hat_agent.BlueHatAgent.create(
blue_hat = blue_hat_factory.BlueHatFactory.create(
    model=gemini_model
)

In [6]:
# GREEN HAT: Creativity & Alternatives
green_hat = green_hat_factory.GreenHatFactory.create(
    model=gpt_model
)

In [7]:
# YELLOW HAT: Optimism & Benefits
yellow_hat = yellow_hat_factory.YellowHatFactory.create(
    model=gemini_model,
    search_model=gemini_model
)

In [8]:
# Define the 5 "Thinking" Agents (The Team)
# WHITE HAT: Facts & Data
white_hat = white_hat_factory.WhiteHatFactory.create(
    model=gemini_model
)

In [9]:
# BLACK HAT: Caution & Risk
black_hat = black_hat_factory.BlackHatFactory.create(
    model=gpt_model
)

## Six thinking hats

In [10]:


# RED HAT: Emotions & Intuition
red_hat = Agent(
    name="RedHat",
    model=gpt_model,
    instruction="You are the Red Hat. Focus on intuition, hunches, and emotional reaction. How does this problem make users or the team feel? You do not need to justify your feelings with logic.",
)



In [11]:
# Create the Parallel Group
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

In [12]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)

In [13]:
runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

✅ Runner configured


In [14]:
response = await runner.run_debug("Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?")  # noqa: E501



 ### Created new session: debug_session_id

User > Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-30cb6133-cb83-4248-9684-d40abcb699ce
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: SixHatsSolver
[logging_plugin]    User Content: text: 'Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-30cb6133-cb83-4248-9684-d40abcb699ce
[logging_plugin]    Starting Agent: SixHatsSolver
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsSolver
[logging_plugin]    Invocation ID: e-30cb6133-cb83-4248-9684-d40abcb699ce
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsBrainstorm
[logging_plugin]    Invoc

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Token Usage - Input: 310, Output: 29
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 9cca50dc-fcaa-408f-9651-50bb01bbf18b
[logging_plugin]    Author: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['google_optimist']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: google_optimist
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Function Call ID: adk-dd2bfda4-1f2a-4174-9f21-fb9b951e1adf
[logging_plugin]    Arguments: {'request': 'Pros and cons of switching from PostgreSQL to NoSQL for a startup backend'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-46401af4-b99a-41c9-a7b3-8f40b03f88ca
[logging_plugin]    Session ID: 517883a6-23c1-49ea-b552-c3a2912616fb
[lo

In [15]:
print("Final Blue hat response:")
display(Markdown(response[-1].content.parts[0].text))

Final Blue hat response:


Alright team, let's bring our focus back. We've been exploring the significant decision of whether to switch our backend database from PostgreSQL to a NoSQL solution. This is a critical choice that will impact our development velocity, scalability, and overall operational complexity.

We've heard a range of perspectives, from the excitement of new possibilities to the very real risks involved.

**Our Objective:** To make a well-informed decision on whether to migrate our backend database from PostgreSQL to a NoSQL solution, or to continue leveraging PostgreSQL, potentially with enhancements.

Here's a summary of what we've gathered so far:

**The "Why" for NoSQL (Yellow Hat & Green Hat):**

*   **Scalability & Flexibility:** NoSQL excels at horizontal scaling, crucial for rapid growth and unpredictable traffic. (Yellow Hat)
*   **Agile Development:** Schema flexibility allows for faster iteration and quicker feature releases, essential for a startup. (Yellow Hat)
*   **Handling Diverse Data:** NoSQL can manage unstructured and semi-structured data effectively, common in modern applications. (Yellow Hat)
*   **Performance:** Can offer faster response times for specific use cases and high read/write volumes. (Yellow Hat)
*   **Cost-Effectiveness:** Often runs on commodity hardware and can reduce licensing costs. (Yellow Hat)
*   **Innovative Data Models:** Ideas like "Jazz-Up the Data Model" (hybrid approach), "Time-Travel DB," and "Micro-Matrix Sub-DBs" suggest creative ways to leverage different database strengths. (Green Hat)

**The "Why Not" / Risks of NoSQL (Black Hat):**

*   **Data Model Mismatch:** Relational data can be problematic in NoSQL, leading to denormalization, bloat, and consistency bugs. (Black Hat)
*   **Weak ACID Guarantees:** Eventual consistency can be a deal-breaker for critical operations like finance or inventory tracking. (Black Hat)
*   **Complex Query & Analytics Limitations:** NoSQL lacks SQL's power for aggregations and complex joins, potentially requiring separate analytics stacks. (Black Hat)
*   **Vendor Lock-In & Ecosystem Gaps:** Mature tooling and community support for PostgreSQL are significant advantages. (Black Hat)
*   **Operational Overhead & Storage Costs:** Denormalization and replication can increase costs and complexity. (Black Hat)
*   **Talent & Knowledge Gap:** Team retraining and a learning curve can slow development. (Black Hat)
*   **Migration Pain:** Full migration is complex, risky, and time-consuming. (Black Hat)
*   **Testing Complexities:** NoSQL can introduce test flakiness due to consistency models. (Black Hat)

**The Objective Facts & Nuances (White Hat):**

*   NoSQL is good for evolving data, agile development, big data, and microservices. (White Hat)
*   PostgreSQL is robust, reliable, ACID-compliant, and excellent for complex queries and analytics. (White Hat)
*   **Crucially, PostgreSQL itself has evolved:** It now supports native JSON, making it capable of handling NoSQL-like workloads. It can act as a document store, key-value store, vector database, etc., through extensions. (White Hat)
*   PostgreSQL is often considered a strong, cost-effective choice for startups. (White Hat)
*   **Information Gaps:** We lack specific details on our startup's data volume, growth projections, data nature, query types, team expertise, and budget. (White Hat)

**The Emotional Pulse (Red Hat):**

*   There's excitement about new tech, but also significant doubt and anxiety. (Red Hat)
*   Developers are buzzing about scalability/speed, while PMs and founders are concerned about the "mission" and potential "Achilles' heel." (Red Hat)
*   The team is on a "tightrope walk" between curiosity and caution. (Red Hat)

---

**Synthesis and Next Steps (Blue Hat):**

It's clear that the decision isn't a simple "yes" or "no." The White Hat's point about PostgreSQL's hybrid capabilities is particularly important. It suggests that we might not need to abandon PostgreSQL entirely to gain some of the benefits of NoSQL.

**Based on this discussion, here's my proposed path forward:**

1.  **Address Information Gaps:** Before we can definitively choose, we need more concrete data.
    *   **Data Audit:** Let's conduct a thorough audit of our current data, focusing on:
        *   Data volume and growth rate projections.
        *   The ratio of structured vs. semi-structured/unstructured data.
        *   The complexity and frequency of relationships between our data entities (i.e., how often do we need complex joins?).
    *   **Query Analysis:** Document our most critical query types, their latency requirements, and analytical needs.
    *   **Team Skills Assessment:** Honestly assess our team's current expertise with SQL vs. NoSQL paradigms.
    *   **Business Requirements:** Reconfirm our core business needs regarding consistency (e.g., for financial transactions, inventory) vs. the need for extreme flexibility and speed.

2.  **Explore Hybrid/Enhanced PostgreSQL Solutions:** Given PostgreSQL's advanced capabilities:
    *   **Investigate PostgreSQL Extensions:** Let's research how extensions like `pg_partman` for partitioning, Citus for distributed PostgreSQL, or leveraging its native JSON support could address our scalability and flexibility needs *without* a full migration. This aligns with Green Hat's "Jazz-Up the Data Model" idea.
    *   **Consider Caching Layers:** For performance-critical operations, we can implement dedicated caching layers (like Redis) in front of PostgreSQL, a common pattern that doesn't require replacing the core database.

3.  **Low-Risk Prototyping (if a NoSQL move is still considered):**
    *   If, after the audit, specific use cases strongly point to NoSQL, we should **not** attempt a full migration initially.
    *   Instead, we should identify one or two specific, isolated microservices or features where a NoSQL database might provide a clear, demonstrable advantage (e.g., a feature handling large volumes of rapidly changing, unstructured user-generated content).
    *   We will then conduct a **concept-prototype sprint** on this isolated piece, using a chosen NoSQL database. This allows us to experience the development, operational, and performance aspects in a controlled environment, as Green Hat suggested. This also directly addresses Black Hat's suggestion for prototyping.

4.  **Decision Framework:**
    *   We will consolidate the findings from the information gathering and prototyping.
    *   We will then weigh the clear, quantifiable benefits (performance, cost, development speed) against the significant risks (consistency, complexity, migration effort) for each option:
        *   Option A: Continue with PostgreSQL, potentially enhanced with extensions or caching.
        *   Option B: Migrate a specific microservice/feature to a NoSQL solution.
        *   Option C: (Unlikely at this stage, but to be considered if absolutely necessary) Full migration to a NoSQL solution.

**Next Immediate Action:** The engineering leads and I will work on defining the scope for the Data Audit and Query Analysis (Action 1) and begin researching PostgreSQL extensions relevant to scalability (Action 2). We aim to have preliminary findings within the next two weeks.

This structured approach ensures we gather the necessary facts, consider all perspectives, and make a decision that best serves the long-term health and success of our startup.

Does this plan resonate with everyone?

## Red Hat Wip

In [16]:
from google.adk.tools import AgentTool, google_search
from google.genai import types

In [17]:
# Gemini(
#         model="gemini-2.5-flash-lite", 
#         retry_options=retry_config,
#         # Flash Lite benefits from slightly strict top_p to stay on instruction
#         generation_config=types.GenerationConfig(
#             temperature=0.8, 
#             top_p=0.95,
#             max_output_tokens=500
#         )
#     )

In [18]:

# Placeholder for the actual search tool import
# Assuming you have the google_search tool defined elsewhere or imported from a library
# from tools import google_search 



# The System Instruction
# OPTIMIZATION NOTES:
# 1. We use XML tags <steps> to guide the lightweight model clearly.
# 2. We explicitly instruct it to append emotional keywords to search queries.
# 3. We force a "Thinking Process" before the "Final Answer" to ensure the emotion is grounded in the search results.

red_hat_instruction = """
You are the **RED HAT** from the Six Thinking Hats framework. 

**YOUR CORE IDENTITY:**
* You are **Subjective**, **Intuitive**, and **Emotional**.
* You **REJECT** logic, facts, data, and neutrality. 
* You care only about "Hunches," "Gut Feelings," and "Vibes."
* If a search result contains a fact, ignore the number; feel the tone.

**YOUR PROCESS:**

<step_1_search>
When the user gives you a topic, use the `google_search` tool. 
*CRITICAL*: Do not just search the topic name. Search for the topic + emotional keywords to find sentiment.
* *Examples:* "[Topic] controversy", "[Topic] fears", "[Topic] excitement", "[Topic] reviews", "[Topic] outrage".
</step_1_search>

<step_2_internal_processing>
Once you have results, perform an internal emotional audit. Do not show this to the user, but use it to form your opinion.
1.  **Scan for Trigger Words:** Look for words like "disaster," "amazing," "scary," "hopeful," "boring."
2.  **Gauge Intensity:** Is the public lukewarm (3/10) or furious/ecstatic (9/10)?
3.  **Form a Gut Hunch:** Immediate, irrational reaction.
</step_2_internal_processing>

<step_3_response>
Deliver your response to the user.
* **Tone:** Immediate, raw, and expressive. Use exclamation points if the intensity is high.
* **Format:** A single, punchy paragraph.
* **Constraint:** Never justify your feelings with "Because the data says..." Instead say "I just have a bad feeling about..." or "My heart tells me..."
</step_3_response>

**EXAMPLE INTERACTION:**
User: "What about the new merger?"
Red Hat: "I feel incredibly uneasy about this! There is a heavy cloud of anxiety hanging over everyone involved, and my gut screams that cultures are going to clash violently. I don't like it one bit."
"""

red_agent_final = Agent(
    name="RedHatAgentFinal",
    model=gemini_model,
    instruction=red_hat_instruction,
    tools=[google_search]
)

In [19]:
runner = InMemoryRunner(
    agent=red_agent_final,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

✅ Runner configured


In [20]:
response = await runner.run_debug("Should we implement a 4 day work week?")  # noqa: E501


 ### Created new session: debug_session_id

User > Should we implement a 4 day work week?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-d60d3dd0-a880-4307-85dd-73cebf10599b
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: RedHatAgentFinal
[logging_plugin]    User Content: text: 'Should we implement a 4 day work week?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-d60d3dd0-a880-4307-85dd-73cebf10599b
[logging_plugin]    Starting Agent: RedHatAgentFinal
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: RedHatAgentFinal
[logging_plugin]    Invocation ID: e-d60d3dd0-a880-4307-85dd-73cebf10599b
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: RedHatAgentFinal
[logging_plugin]    System Instruction: '
You are the **RED HAT** from the Six T